# Head Pose Estimation

## Introduction
In many applications, we need to know how the head is tilted with respect to a camera. In a virtual reality application, for example, one can use the pose of the head to render the right view of the scene. In a driver assistance system, a camera looking at a driver’s face in a vehicle can use head pose estimation to see if the driver is paying attention to the road. And of course one can use head pose based gestures to control a hands-free application / game. For example, yawing your head left to right can signify a NO. 
## What is pose estimation?
In computer vision, the pose of an object refers to its relative orientation and position with respect to a camera. You can change the pose by either moving the object with respect to the camera, or the camera with respect to the object.
The pose estimation problem described in this tutorial is often referred to as Perspective-n-Point problem or PNP in computer vision jargon. As we shall see in the following sections in more detail, in this problem the goal is to find the pose of an object when we have a calibrated camera, and we know the locations of n 3D points on the object and the corresponding 2D projections in the image.

## How to mathematically represent camera motion ?

A 3D rigid object has only two kinds of motions with respect to a camera:
1. **Translation:** Moving the camera from its current 3D location *(X, Y, Z)* to a new 3D location *(X', Y', Z')* is called translation. As you can see translation has 3 degrees of freedom — you can move in the X, Y or Z direction. Translation is represented by a vector *t* which is equal to *(X'-X, Y'-Y, Z'-Z)*.
2. **Rotation:** You can also rotate the camera about the X, Y and Z axes. A rotation, therefore, also has three degrees of freedom. There are many ways of representing rotation. You can represent it using **Euler angles**( roll, pitch and yaw ), a 3X3 **rotation matrix**, or a **direction of rotation (i.e. axis ) and angle**.

So, estimating the pose of a 3D object means finding 6 numbers — three for translation and three for rotation.

## What do you need for pose estimation ?
<center> <a href="https://www.learnopencv.com/wp-content/uploads/2018/01/opcv4face-w4-m8-3d.jpg"><img src = "https://www.learnopencv.com/wp-content/uploads/2018/01/opcv4face-w4-m8-3d.jpg"/></a></center>

To calculate the 3D pose of an object in an image you need the following information:
+ **2D coordinates of a few points :** You need the 2D (x, y) locations of a few points in the image. In the case of a face, you could choose the corners of the eyes, the tip of the nose, corners of the mouth etc. Dlib’s facial landmark detector provides us with many points to choose from. In this tutorial, we will use the tip of the nose, the chin, the left corner of the left eye, the right corner of the right eye, the left corner of the mouth, and the right corner of the mouth.
+ **3D locations of the same points :** You also need the 3D location of the 2D feature points. You might be thinking that you need a 3D model of the person in the photo to get the 3D locations. Ideally yes, but in practice, you don’t. A generic 3D model will suffice. Where do you get a 3D model of a head from ? Well, you really don’t need a full 3D model. You just need the 3D locations of a few points in some arbitrary reference frame. In this tutorial, we are going to use the following 3D points.
    -    Tip of the nose : ( 0.0, 0.0, 0.0)
    -    Chin : ( 0.0, -330.0, -65.0)
    -    Left corner of the left eye : (-225.0, 170.0, -135.0)
    -    Right corner of the right eye : ( 225.0, 170.0, -135.0)
    - Left corner of the mouth : (-150.0, -150.0, -125.0)
    -    Right corner of the mouth : (150.0, -150.0, -125.0)

+    **Intrinsic parameters of the camera**. As mentioned before, in this problem the camera is assumed to be calibrated. In other words, you need to know the focal length of the camera, the optical center in the image and the radial distortion parameters. So you need to calibrate your camera. There is a trick to solve this problem too. We can approximate the optical center by the center of the image, approximate the focal length by the width of the image in pixels and assume that radial distortion does not exist.


## How do pose estimation algorithms work ?
There are several algorithms for pose estimation. The first known algorithm dates back to 1841. It is beyond the scope of this post to explain the details of these algorithms but here is a general idea.

There are three coordinate systems in play here. The 3D coordinates of the various facial features shown above are in world coordinates. If we knew the rotation and translation (i.e. pose), we could transform the 3D points in **world coordinates** to 3D points in **camera coordinates**. The 3D points in camera coordinates can be projected onto the image plane (i.e. **image coordinate system**) using the intrinsic parameters of the camera (focal length, optical center etc.).
<center> <a href="https://www.learnopencv.com/wp-content/uploads/2018/01/opcv4face-w4-m8-intrinsicCameraParameters.jpg"><img src = "https://www.learnopencv.com/wp-content/uploads/2018/01/opcv4face-w4-m8-intrinsicCameraParameters.jpg"/></a></center>

Let’s dive into the image formation equation to understand how these above coordinate systems work. In the figure above, *o* is the center of the camera and plane shown in the figure is the image plane. We are interested in finding out what equations govern the projection p of the 3D point P onto the image plane.

Let’s assume we know the location(U, V, W)of a 3D point Pin World Coordinates. If we know the rotation R(3×3 matrix) and translation t (3×1 vector), of the world coordinates with respect to the camera coordinates, we can calculate the location (X, Y, Z) of the point Pin the camera coordinate system using the following equation.
$$
\begin{bmatrix} X \\ Y \\Z  \end{bmatrix}  = R  \begin{bmatrix} U \\ V \\W  \end{bmatrix}+t  
$$
$$
=>\begin{bmatrix} X \\ Y \\Z  \end{bmatrix}  = \begin{bmatrix} &R&|&t&\end{bmatrix}  \begin{bmatrix} U \\ V \\W \\1  \end{bmatrix}  
$$ 
                                                     
In expanded form, the above equation looks like this
$$
=>\begin{bmatrix} X \\ Y \\Z  \end{bmatrix}  = \begin{bmatrix} r_{00} &r_{01} &r_{02} &t_x \\r_{10} &r_{11} &r_{12} &t_y \\r_{20} &r_{21} &r_{22} &t_z\end{bmatrix}  \begin{bmatrix} U \\ V \\W \\1  \end{bmatrix}  
$$                                              
If you have ever taken a Linear Algebra class, you will recognize that if we knew sufficient number of point correspondences (i.e. $(X, Y, Z)$ and $(U, V, W)$), the above is a linear system of equations where the $r_{ij}$ and $( t_x, t_y, t_z )$ are unknowns and you can trivially solve for the unknowns.

As you will see in the next section, we know $(X, Y, Z)$ only up to an unknown scale, and so we do not have a simple linear system.

## Direct Linear Transform
We do know many points on the 3D model $(U, V, W)$ ), but we do not know $(X, Y, Z)$. We only know the location of the 2D points ( i.e. $(x, y)$ ). In the absence of radial distortion, the coordinates $(x, y)$ of point p in the image coordinates is given by
$$
\begin{bmatrix} X \\ Y \\1  \end{bmatrix}=s\begin{bmatrix} f_x &0 &c_x \\ 0 &f_y &c_y \\0 &0 &1  \end{bmatrix}\begin{bmatrix} X \\ Y \\Z  \end{bmatrix}
$$

where,  $f_x$ and  $f_y$ are the focal lengths in the x and y directions, and ( $c_x,  c_y$ ) is the optical center. Things get slightly more complicated when radial distortion is involved and for the purpose of simplicity I am leaving it out.

What about that $s$ in the equation ? It is an unknown scale factor. It exists in the equation due to the fact that in any image we do not know the depth. If you join any point $P$ in 3D to the center $o$ of the camera, the point $p$, where the ray intersects the image plane is the image of $P$. Note that all the points along the ray joining the center of the camera and point $P$ produce the same image. In other words, using the above equation, you can only obtain $(X, Y, Z)$ up to a scale $s$.

Now this messes up equation 2 because it is no longer the nice linear equation we know how to solve. Our equation looks more like
$$
s\begin{bmatrix} X \\ Y \\Z  \end{bmatrix}  = \begin{bmatrix} r_{00} &r_{01} &r_{02} &t_x \\r_{10} &r_{11} &r_{12} &t_y \\r_{20} &r_{21} &r_{22} &t_z\end{bmatrix}  \begin{bmatrix} U \\ V \\W \\1  \end{bmatrix}  
$$
Fortunately, the equation of the above form can be solved using some algebraic wizardry using a method called **Direct Linear Transform (DLT)**. You can use DLT any time you find a problem where the equation is almost linear but is off by an unknown scale.

## Levenberg-Marquardt Optimization
The DLT solution mentioned above is not very accurate because of the following reasons . First, rotation has three degrees of freedom but the matrix representation used in the DLT solution has 9 numbers. There is nothing in the DLT solution that forces the estimated $3x3$ matrix to be a rotation matrix. More importantly, the DLT solution does not minimize the correct objective function. Ideally, we want to minimize the reprojection error that is described below.

As shown in the equations 2 and 3, if we knew the right pose ( $R$ and $t$ ), we could predict the 2D locations of the 3D facial points on the image by projecting the 3D points onto the 2D image. In other words, if we knew $R$ and $t$ we could find the point $p$ in the image for every 3D point $P$.

We also know the 2D facial feature points ( using Dlib or manual clicks ). We can look at the distance between projected 3D points and 2D facial features. When the estimated pose is perfect, the 3D points projected onto the image plane will line up almost perfectly with the 2D facial features. When the pose estimate is incorrect, we can calculate a re-projection error measure — the sum of squared distances between the projected 3D points and 2D facial feature points.

As mentioned earlier, an approximate estimate of the pose ($R$ and $t$) can be found using the DLT solution. A naive way to improve the DLT solution would be to randomly change the pose ( $R$ and $t$) slightly and check if the reprojection error decreases. If it does, we can accept the new estimate of the pose. We can keep perturbing $R$ and $t$ again and again to find better estimates. While this procedure will work, it will be very slow. Turns out there are principled ways to iteratively change the values of $R$ and $t$ so that the reprojection error decreases. One such method is called Levenberg-Marquardt optimization. Check out more details on Wikipedia article on [Levenberg-Marquardt algorithm](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm).

## OpenCV solvePnP
In OpenCV the function **solvePnP** and **solvePnPRansac** can be used to estimate pose.

solvePnP implements several algorithms for pose estimation which can be selected using the parameter flag. By default it uses the flag **SOLVEPNP_ITERATIVE** which is essentially the DLT solution followed by Levenberg-Marquardt optimization. **SOLVEPNP_P3P** uses only 3 points for calculating the pose and it should be used only when using solvePnPRansac.

In OpenCV 3, two new methods have been introduced — **SOLVEPNP_DLS** and **SOLVEPNP_UPNP**. The interesting thing about SOLVEPNP_UPNP is that it tries to estimate camera internal parameters also.

```python

retval, rvec, tvec	=	cv.solvePnP(	objectPoints, imagePoints, cameraMatrix, distCoeffs[, rvec[, tvec[, useExtrinsicGuess[, flags]]]]	)

```

Where,

- **`objectPoints`** - Array of object points in the object coordinate space, Nx3 1-channel or 1xN/Nx1 3-channel, where N is the number of points. vector<Point3f> can be also passed here.
- **`imagePoints`** - Array of corresponding image points, Nx2 1-channel or 1xN/Nx1 2-channel, where N is the number of points. vector<Point2f> can be also passed here.
- **`cameraMatrix`** - Input camera matrix.
- **`distCoeffs`** - Input vector of distortion coefficients ($k_1$,$k_2$,$p_1$,$p_2$[,$k_3$[,$k_4$,$k_5$,$k_6$[,$s_1$,$s_2$,$s_3$,$s_4$[,$\tau_x$,$\tau_y$]]]]) of 4, 5, 8, 12 or 14 elements. If the vector is NULL/empty, the zero distortion coefficients are assumed.
- **`rvec`** - Output rotation vector that, together with tvec , brings points from the model coordinate system to the camera coordinate system.
- **`tvec`** - Output translation vector.
- **`useExtrinsicGuess`** - Parameter used for SOLVEPNP_ITERATIVE. If true (1), the function uses the provided rvec and tvec values as initial approximations of the rotation and translation vectors, respectively, and further optimizes them.
- **`flags`** - Method for solving a PnP problem:
  - **`SOLVEPNP_ITERATIVE`** - Iterative method is based on Levenberg-Marquardt optimization. In this case the function finds such a pose that minimizes reprojection error, that is the sum of squared distances between the observed projections imagePoints and the projected (using projectPoints ) objectPoints .
  - **`SOLVEPNP_P3P`** - Method is based on the paper of X.S. Gao, X.-R. Hou, J. Tang, H.-F. Chang "Complete Solution Classification for the Perspective-Three-Point Problem". In this case the function requires exactly four object and image points.
  - **`SOLVEPNP_AP3P`** - Method is based on the paper of T. Ke, S. Roumeliotis "An Efficient Algebraic Solution to the Perspective-Three-Point Problem". In this case the function requires exactly four object and image points.
  - **`SOLVEPNP_EPNP`** - Method has been introduced by F.Moreno-Noguer, V.Lepetit and P.Fua in the paper "EPnP: Efficient Perspective-n-Point Camera Pose Estimation".
  - **`SOLVEPNP_DLS`** - Method is based on the paper of Joel A. Hesch and Stergios I. Roumeliotis. "A Direct Least-Squares (DLS) Method for PnP".
  - **`SOLVEPNP_UPNP`** - Method is based on the paper of A.Penate-Sanchez, J.Andrade-Cetto, F.Moreno-Noguer. "Exhaustive Linearization for Robust Camera Pose and Focal Length Estimation". In this case the function also estimates the parameters fx and fy assuming that both have the same value. Then the cameraMatrix is updated with the estimated focal length.
  - **`SOLVEPNP_AP3P`** - Method is based on the paper of Tong Ke and Stergios I. Roumeliotis. "An Efficient Algebraic Solution to the Perspective-Three-Point Problem". In this case the function requires exactly four object and image points.

## OpenCV solvePnPRansac

**solvePnPRansac** is very similar to **solvePnP** except that it uses **Random Sample Consensus (RANSAC)** for robustly estimating the pose.

Using RANSAC is useful when you suspect that a few data points are extremely noisy. For example, consider the problem of fitting a line to 2D points. This problem can be solved using linear least squares where the distance of all points from the fitted line is minimized. Now consider one bad data point that is wildly off. This one data point can dominate the least squares solution and our estimate of the line would be very wrong. In RANSAC, the parameters are estimated by randomly selecting the minimum number of points required. In a line fitting problem, we randomly select two points from all data and find the line passing through them. Other data points that are close enough to the line are called inliers. Several estimates of the line are obtained by randomly selecting two points, and the line with the maximum number of inliers is chosen as the correct estimate.

The usage of **solvePnPRansac** is shown below and parameters specific to solvePnPRansac are explained:

```python
retval, rvec, tvec, inliers	=	cv.solvePnPRansac(	objectPoints, imagePoints, cameraMatrix, distCoeffs[, rvec[, tvec[, useExtrinsicGuess[, iterationsCount[, reprojectionError[, confidence[, inliers[, flags]]]]]]]]	)
```

Where, 

- **`objectPoints`** - Array of object points in the object coordinate space, Nx3 1-channel or 1xN/Nx1 3-channel, where N is the number of points. vector<Point3f> can be also passed here.
- **`imagePoints`** - Array of corresponding image points, Nx2 1-channel or 1xN/Nx1 2-channel, where N is the number of points. vector<Point2f> can be also passed here.
- **`cameraMatrix`** - Input camera matrix
- **`distCoeffs`** - Input vector of distortion coefficients ($k_1$,$k_2$,$p_1$,$p_2$[,$k_3$[,$k_4$,$k_5$,$k_6$[,$s_1$,$s_2$,$s_3$,$s_4$[,$\tau_x$,$\tau_y$]]]]) of 4, 5, 8, 12 or 14 elements. If the vector is NULL/empty, the zero distortion coefficients are assumed.
rvec	Output rotation vector that, together with tvec , brings points from the model coordinate system to the camera coordinate system.
- **`tvec`** - Output translation vector.
- **`useExtrinsicGuess`** - Parameter used for SOLVEPNP_ITERATIVE. If true (1), the function uses the provided rvec and tvec values as initial approximations of the rotation and translation vectors, respectively, and further optimizes them.
- **`iterationsCount`** - Number of iterations.
- **`reprojectionError`** - Inlier threshold value used by the RANSAC procedure. The parameter value is the maximum allowed distance between the observed and computed point projections to consider it an inlier.
- **`confidence`** - The probability that the algorithm produces a useful result.
- **`inliers`** - Output vector that contains indices of inliers in objectPoints and imagePoints .
- **`flags`** - Method for solving a PnP problem

## OpenCV POSIT

OpenCV used to have a pose estimation algorithm called POSIT. It is still present in the C API ([cvPosit](https://github.com/opencv/opencv/blob/2f4e38c8313ff313de7c41141d56d945d91f47cf/modules/calib3d/src/posit.cpp)), but is not part of the C++ API. POSIT assumes a scaled orthographic camera model and therefore you do not need to supply a focal length estimate. This function is now obsolete and I would recommend using one of the algorithms implemented in **solvePnp**.

### OpenCV Pose Estimation Code

Now let’s dive into Python code for head pose estimation in a single image.

The locations of facial feature points are hard coded and if you want to use your own image, you will need to change the vector image_points.

In [7]:
import cv2
import dlib
import numpy as np
from renderFace import renderFace
import matplotlib.pyplot as plt
from dataPath import DATA_PATH
from dataPath import MODEL_PATH
import sys
%matplotlib inline

In [3]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (6.0,6.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [5]:
SKIP_FRAMES = 20
RESIZE_HEIGHT = 320
PREDICTOR_PATH = MODEL_PATH + "shape_predictor_68_face_landmarks.dat"

We write some functions which return 3D model points in an arbitrary frame of reference as assumed by us, 2D points of selected landmarks and camera parameters.


In [6]:
# 3D Model Points of selected landmarks in an 
# arbitrary frame of reference
def get3dModelPoints():
  modelPoints = [[0.0, 0.0, 0.0],
                 [0.0, -330.0, -65.0],
                 [-225.0, 170.0, -135.0],
                 [225.0, 170.0, -135.0],
                 [-150.0, -150.0, -125.0],
                 [150.0, -150.0, -125.0]]
  return np.array(modelPoints, dtype=np.float64)


# 2D landmark points from all landmarks
def get2dImagePoints(shape):
  imagePoints = [[shape.part(30).x, shape.part(30).y],
                 [shape.part(8).x, shape.part(8).y],
                 [shape.part(36).x, shape.part(36).y],
                 [shape.part(45).x, shape.part(45).y],
                 [shape.part(48).x, shape.part(48).y],
                 [shape.part(54).x, shape.part(54).y]]
  return np.array(imagePoints, dtype=np.float64)


# Camera Matrix from focal length and focal center
def getCameraMatrix(focalLength, center):
  cameraMatrix = [[focalLength, 0, center[0]],
                  [0, focalLength, center[1]],
                  [0, 0, 1]]
  return np.array(cameraMatrix, dtype=np.float64)

Initialize VideoCapture object, calculate resize scale based on dimensions of first frame and initialize face detector and landmarks predictor objects.

In [ ]:
try:
  # Create a VideoCapture object
  cap = cv2.VideoCapture(0)

  # Check if OpenCV is able to read feed from camera
  if (cap.isOpened() is False):
    print("Unable to connect to camera")
    sys.exit(0)

  # Just a place holder. Actual value calculated after 100 frames.
  fps = 30.0

  # Get first frame
  ret, im = cap.read()

  # We will use a fixed height image as input to face detector
  if ret == True:
    height = im.shape[0]
    # calculate resize scale
    RESIZE_SCALE = float(height)/RESIZE_HEIGHT
    size = im.shape[0:2]
  else:
    print("Unable to read frame")
    sys.exit(0)

  # Load face detection and pose estimation models
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor(PREDICTOR_PATH)

  # initiate the tickCounter
  t = cv2.getTickCount()
  count = 0

  # Grab and process frames until the main window is closed by the user.
  while(True):

    # start tick counter if count is zero
    if count==0:
      t = cv2.getTickCount()

    # Grab a frame
    ret, im = cap.read()

    # create imSmall by resizing image by resize scale
    imSmall= cv2.resize(im, None, fx = 1.0/RESIZE_SCALE, fy = 1.0/RESIZE_SCALE, interpolation = cv2.INTER_LINEAR)

    # Process frames at an interval of SKIP_FRAMES.
    # This value should be set depending on your system hardware
    # and camera fps.
    # To reduce computations, this value should be increased
    if (count % SKIP_FRAMES == 0):

      # Detect faces
      faces = detector(cv2.cvtColor(imSmall, cv2.COLOR_BGR2RGB), 0)

    # get 3D model points
    modelPoints = get3dModelPoints()

    # Iterate over faces
    for face in faces:
      # Since we ran face detection on a resized image,
      # we will scale up coordinates of face rectangle
      newRect = dlib.rectangle(int(face.left() * RESIZE_SCALE),
                               int(face.top() * RESIZE_SCALE),
                               int(face.right() * RESIZE_SCALE),
                               int(face.bottom() * RESIZE_SCALE))

      # Find face landmarks by providing reactangle for each face
      shape = predictor(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), newRect)

      # Draw landmarks over face
      renderFace(im, shape)

      # get 2D landmarks from Dlib's shape object
      imagePoints = get2dImagePoints(shape)

      # Camera parameters
      rows, cols, ch = im.shape
      focalLength = cols
      cameraMatrix = getCameraMatrix(focalLength, (rows/2, cols/2))

      # Assume no lens distortion
      distCoeffs = np.zeros((4, 1), dtype=np.float64)

      # calculate rotation and translation vector using solvePnP
      success, rotationVector, translationVector = cv2.solvePnP(modelPoints, imagePoints, cameraMatrix, distCoeffs)

      # Project a 3D point (0, 0, 1000.0) onto the image plane.
      # We use this to draw a line sticking out of the nose
      noseEndPoints3D = np.array([[0, 0, 1000.0]], dtype=np.float64)
      noseEndPoint2D, jacobian = cv2.projectPoints(noseEndPoints3D, rotationVector, translationVector, cameraMatrix, distCoeffs)

      # points to draw line
      p1 = (int(imagePoints[0, 0]), int(imagePoints[0, 1]))
      p2 = (int(noseEndPoint2D[0, 0, 0]), int(noseEndPoint2D[0, 0, 1]))

      # draw line using points P1 and P2
      cv2.line(im, p1, p2, (110, 220, 0), thickness=2, lineType=cv2.LINE_AA)
      # Print actual FPS
      cv2.putText(im, "fps: {}".format(fps), (50, size[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

      # Resize image for display
      imDisplay = cv2.resize(im, None, fx=0.5, fy=0.5)
      cv2.imshow("webcam Head Pose", imDisplay)

      # WaitKey slows down the runtime quite a lot
      # So check every 15 frames
      if (count % 15 == 0):
        key = cv2.waitKey(1) & 0xFF

        # Stop the program.
        if key==27:  # ESC
          # If ESC is pressed, exit.
          sys.exit()

      # Calculate actual fps
      # increment frame counter
      count = count + 1
      # calculate fps at an interval of 100 frames
      if (count == 100):
        t = (cv2.getTickCount() - t)/cv2.getTickFrequency()
        fps = 100.0/t
        count = 0

  cap.release()
  cv2.destroyAllWindows()

except Exception as e:
  print(e)

## Getting Roll, Pitch and Yaw angles from rotation vector

Once you get the rotationVector as shown in the code above, you can use [**Rodrigues**](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html?highlight=euler#rodrigues) function to get the rotation matrix and again use the rotation matrix to get the euler angles using the [**RQDecomposition**](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html?highlight=euler#rqdecomp3x3) function. Sample code for doing this is given below

```python
rmat,jac = cv2.Rodrigues(rotationVector)
angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
print(angles)
```

You should get something like
```
(-169.12891777767527, 11.408486358348176, -6.712484655823574)
```

These are the rotation angles along the X, Y, and Z axes.

## Head Pose with Stabilization
We have also provided code : `webcamHeadPoseWithStabilization.py` for finding head pose with landmark stabilization ( discussed in previous module ).

# References and Further Reading
https://en.wikipedia.org/wiki/Perspective-n-Point

http://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html

https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm

https://en.wikipedia.org/wiki/Direct_linear_transformation

https://github.com/natanielruiz/deep-head-pose

https://stackoverflow.com/questions/12933284/rodrigues-into-eulerangles-and-vice-versa